In [2]:
import os

In [3]:
%pwd

'e:\\Weather_prediction\\research'

In [4]:
os.chdir("E:/Weather_prediction")

In [5]:
%pwd

'E:\\Weather_prediction'

In [6]:
import pandas as pd

data = pd.read_csv(r"E:\Weather_prediction\artifacts\data_ingestion\weather.csv")
data.head()

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,precipitation_hours,temperature_2m_mean,apparent_temperature_mean,cloud_cover_mean,cloud_cover_max,cloud_cover_min,sunshine_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_gusts_10m_mean,wind_speed_10m_mean,relative_humidity_2m_max,relative_humidity_2m_min,relative_humidity_2m_mean
0,2020-08-14 18:30:00+00:00,27.244,22.144,6.900000,6.900000,18.0,23.887749,26.387281,99.708336,100.0,97.0,11183.3430,23.210928,45.00,31.905000,15.910668,94.962425,72.612100,88.235960
1,2020-08-15 18:30:00+00:00,27.794,22.194,11.599998,11.599998,17.0,24.239832,27.173525,99.666664,100.0,98.0,3441.2383,22.288042,42.12,29.174997,14.512328,95.538380,73.150450,88.800720
2,2020-08-16 18:30:00+00:00,23.994,22.544,65.499990,65.499990,20.0,23.108583,26.037016,99.958336,100.0,99.0,0.0000,18.671474,34.92,28.095001,14.444237,96.131600,92.172400,94.593506
3,2020-08-17 18:30:00+00:00,24.444,22.394,53.500000,53.500000,20.0,23.062752,25.709036,100.000000,100.0,100.0,0.0000,18.806337,36.36,30.494997,15.825063,96.413826,90.527990,94.042330
4,2020-08-18 18:30:00+00:00,27.344,22.144,13.099999,13.099999,20.0,24.085665,26.299780,89.208336,100.0,63.0,27248.9020,24.400460,46.80,35.459995,18.013298,94.964294,72.408295,87.796760


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       1827 non-null   object 
 1   temperature_2m_max         1827 non-null   float64
 2   temperature_2m_min         1827 non-null   float64
 3   precipitation_sum          1827 non-null   float64
 4   rain_sum                   1827 non-null   float64
 5   precipitation_hours        1827 non-null   float64
 6   temperature_2m_mean        1827 non-null   float64
 7   apparent_temperature_mean  1827 non-null   float64
 8   cloud_cover_mean           1827 non-null   float64
 9   cloud_cover_max            1827 non-null   float64
 10  cloud_cover_min            1827 non-null   float64
 11  sunshine_duration          1827 non-null   float64
 12  wind_speed_10m_max         1827 non-null   float64
 13  wind_gusts_10m_max         1827 non-null   float

In [8]:
data.isnull().sum()

date                         0
temperature_2m_max           0
temperature_2m_min           0
precipitation_sum            0
rain_sum                     0
precipitation_hours          0
temperature_2m_mean          0
apparent_temperature_mean    0
cloud_cover_mean             0
cloud_cover_max              0
cloud_cover_min              0
sunshine_duration            0
wind_speed_10m_max           0
wind_gusts_10m_max           0
wind_gusts_10m_mean          0
wind_speed_10m_mean          0
relative_humidity_2m_max     0
relative_humidity_2m_min     0
relative_humidity_2m_mean    0
dtype: int64

In [9]:
data.shape

(1827, 19)

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: Path
    unzip_data_dir: Path
    all_schema: dict

In [11]:
from src.Predict_weather.constants import *
from src.Predict_weather.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            unzip_data_dir= config.unzip_data_dir,
            all_schema= schema
        )
        
        return data_validation_config

In [15]:
import os
from src.Predict_weather import logger

In [16]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_col = list(data.columns)
            all_schema = self.config.all_schema.keys()
            
            for col in all_col:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}")
                        
        except Exception as e:
            raise e

In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-01 21:55:14,785: INFO: common: yaml file: config\config.yml loaded successfully]
[2025-09-01 21:55:14,789: INFO: common: yaml file: params.yml loaded successfully]
[2025-09-01 21:55:14,802: INFO: common: yaml file: schema.yml loaded successfully]
[2025-09-01 21:55:14,805: INFO: common: created directory at: artifacts]
[2025-09-01 21:55:14,805: INFO: common: created directory at: artifacts/data_validation]
